In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Path to the ZIP file
zip_path = "/content/drive/MyDrive/archive.zip"  # Update with your actual file path
extract_path = "/content/dataset"  # Target directory for extraction

# Unzip the file
os.system(f"unzip -q {zip_path} -d {extract_path}")
print(f"✅ Dataset extracted to: {extract_path}")

✅ Dataset extracted to: /content/dataset


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

# Define dataset path
dataset_path = "/content/dataset/Segmented Medicinal Leaf Images"

# Image settings
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation & Loading
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training')

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

# Number of classes
num_classes = len(train_generator.class_indices)

# Function to create models
def create_model(base_model):
    base_model.trainable = False  # Freeze the convolutional base
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define models
models = {
    "VGG-16": create_model(VGG16(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE + (3,))),
}

# Training all models
history_dict = {}
for name, model in models.items():
    print(f"Training {name}...")
    history = model.fit(train_generator, validation_data=val_generator, epochs=10)
    model.save(f"{name}_model.h5")
    history_dict[name] = history

# Plot accuracy comparison
plt.figure(figsize=(12, 6))
for name, history in history_dict.items():
    plt.plot(history.history['accuracy'], label=f'{name} Train')
    plt.plot(history.history['val_accuracy'], label=f'{name} Val')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate models
for name, model in models.items():
    loss, accuracy = model.evaluate(val_generator)
    print(f"{name} - Accuracy: {accuracy * 100:.2f}% | Loss: {loss:.4f}")


Found 1479 images belonging to 30 classes.
Found 356 images belonging to 30 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Training VGG-16...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 44s 624ms/step - accuracy: 0.1140 - loss: 3.3227 - val_accuracy: 0.3006 - val_loss: 2.9053
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 440ms/step - accuracy: 0.3018 - loss: 2.7948 - val_accuracy: 0.3539 - val_loss: 2.4180
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 19s 407ms/step - accuracy: 0.4260 - loss: 2.2752 - val_accuracy: 0.5618 - val_loss: 1.9628
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 416ms/step - accuracy: 0.6403 - loss: 1.8265 - val_accuracy: 0.6770 - val_loss: 1.5980
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 19s 404ms/step - accuracy: 0.7206 - loss: 1.4928 - val_accuracy: 0.7416 - val_loss: 1.3537
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 417ms/step - accuracy: 0.7906 - loss: 1.2271 - val_accuracy: 0.7444 - val_loss: 1.1531
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 19s 408ms/step - accuracy: 0.7827 - loss: 1.0251 - val_accuracy: 0.7444 - val_loss: 1.0297
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step - accuracy: 0.8422 - loss: 0.8390

In [ ]:
!pip install tensorflow
import tensorflow as tf # This line was already in your code
from tensorflow.keras.layers import Dropout # Import Dropout explicitly
from tensorflow.keras.applications import  VGG16
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess

import cv2
import numpy as np
from PIL import Image



In [ ]:
from tensorflow.keras.layers import Dropout

def build_model(base_model, model_name):
    base_model.trainable = False  # Freeze layers initially

    # Unfreeze last layers for fine-tuning
    if model_name in ["VGG-16",]:
        for layer in base_model.layers[-10:]:  # Unfreeze last 10 layers
            layer.trainable = True

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Binary classification
    model = Model(inputs, outputs)

    # Adjust learning rate based on model
    lr = 1e-4 if model_name in ["ResNet-50", "VGG-16"] else 5e-5
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


# Load pre-trained models
vgg16_model = build_model(VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)), "VGG-16")

# Compile models
for model in [ vgg16_model]:
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


def preprocess_image(image_path, model_name):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))

    # Apply correct preprocessing for each model

    if model_name == "VGG-16":
        img = vgg_preprocess(img)


    return np.expand_dims(img, axis=0)


def predict_medicinal_leaf(image_path, models):
    predictions = {}
    for name, model in models.items():
        img = preprocess_image(image_path, name)
        pred = model(img, training=False).numpy()[0][0]  # Ensure correct inference
        predictions[name] = 'Medicinal' if pred > 0.5 else 'Non-Medicinal'
    return predictions




# Example usage:
models = {
    'VGG-16': vgg16_model,

}

# Load trained weights before prediction (replace with actual trained weights path)
# for model_name, model in models.items():
#     model.load_weights(f'./{model_name}_weights.h5')

image_path = '/content/sample_data/neem.jpg'  # Replace with actual path
predictions = predict_medicinal_leaf(image_path, models)
print(predictions)


{'ResNet-50': 'Non-Medicinal', 'VGG-16': 'Non-Medicinal', 'EfficientNet-B0': 'Non-Medicinal', 'MobileNetV3': 'Medicinal'}


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import  VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight

# Dataset Path
dataset_path = "/content/dataset/Segmented Medicinal Leaf Images"

# Image settings
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation & Loading
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Compute Class Weights
class_labels = np.array(train_generator.classes)
class_weights = compute_class_weight('balanced', classes=np.unique(class_labels), y=class_labels)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

In [ ]:

# Model Preprocessing Functions
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess


def preprocess_image(image_path, model_name):
    """Preprocess image according to the specific model's requirements."""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))

    if model_name == "VGG-16":
        img = vgg_preprocess(img)
    return np.expand_dims(img, axis=0)


# Model Creation
def build_model(base_model, model_name):
    """Builds and compiles a model with fine-tuning."""
    base_model.trainable = False  # Initially freeze the model

     # Unfreeze last 10 layers for fine-tuning
    for layer in base_model.layers[-5:]:
        layer.trainable = True

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Binary classification
    model = Model(inputs, outputs)

    # Set learning rate per model
    lr = 1e-4 if model_name in ["VGG-16"] else 5e-5
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Load Pretrained Models
vgg16_model = build_model(VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)), "VGG-16")
# Train Models
models = {

    'VGG-16': vgg16_model,
}

history_dict = {}
for name, model in models.items():
    print(f"Training {name}...")
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=5,
        class_weight=class_weights_dict
    )
    model.save(f"{name}_model.h5")
    history_dict[name] = history

# Plot Accuracy Comparison
plt.figure(figsize=(12, 6))
for name, history in history_dict.items():
    plt.plot(history.history['accuracy'], label=f'{name} Train')
    plt.plot(history.history['val_accuracy'], label=f'{name} Val')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()